In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-03-02 01:28:14.136007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-02 01:28:15.918933: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Define data paths
train_data_dir = './../dataset/image/set2/train'
validation_data_dir = './../dataset/image/set2/test'
test_data_dir = './../dataset/image/set2/test'

# Image dimensions
img_width, img_height = 256, 256
input_shape = (img_width, img_height, 3)

In [3]:
# Define data generators with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

Found 44 images belonging to 2 classes.
Found 126 images belonging to 2 classes.


In [4]:

# Build CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),  # Dropout for regularization
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])




2024-03-02 01:28:21.106109: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14791 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:60:00.0, compute capability: 7.0
2024-03-02 01:28:21.106779: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14791 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB, pci bus id: 0000:61:00.0, compute capability: 7.0
2024-03-02 01:28:21.107333: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14791 MB memory:  -> device: 2, name: Tesla V100-SXM2-16GB, pci bus id: 0000:88:00.0, compute capability: 7.0
2024-03-02 01:28:21.107863: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 14791 MB memory:  -> device: 3, name: Tesla V100-SXM2-16GB, pci bus id

In [5]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // 32
)



Epoch 1/2


2024-03-02 01:28:24.079426: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8900
2024-03-02 01:28:26.464974: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x14dfa8e4c470 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-02 01:28:26.465018: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-03-02 01:28:26.465026: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-03-02 01:28:26.465032: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-03-02 01:28:26.465037: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (3): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-03-02 01:28:26.500329: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir

1/1 [==============================] - 8s 8s/step - loss: 0.6905 - accuracy: 0.5625 - val_loss: 6.5126 - val_accuracy: 0.2292
Epoch 2/2
1/1 [==============================] - 1s 571ms/step - loss: 3.3277 - accuracy: 0.5833 - val_loss: 0.7976 - val_accuracy: 0.2292


In [6]:
# Evaluate the model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

Found 126 images belonging to 2 classes.
4/4 [==============================] - 0s 90ms/step - loss: 0.7836 - accuracy: 0.2698
Test accuracy: 0.2698412835597992
